# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [19]:
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [20]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: gen_ai_gsmoon


### 1.2. policy
- Add the following 4 permissions to the roles confirmed in 1.1
    - AmazonBedrockFullAccess
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess
    - AWSMarketplaceFullAccess
    - AWSCloud9SSMAccessRole

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
Model Activation Window
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
Activate the model to be used and then save.
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

- #### **Please change to `install_complex_pdf = True` when utilizing Complex PDF**.
    - ####  [Note] install_complex_pdf will take about 30 minutes.

In [21]:
install_needed = True
install_complex_pdf = False

In [22]:
import os
import sys
import IPython
import subprocess

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U langchain-community
    !{sys.executable} -m pip install -U langchain_aws
    !{sys.executable} -m pip install -U langchain-experimental
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U pydantic
    !{sys.executable} -m pip install -U python-dotenv
    !{sys.executable} -m pip install -U llama-parse
    !{sys.executable} -m pip install -U pymupdf
    !{sys.executable} -m pip install -U pypdf
    !{sys.executable} -m pip install -U pdf2image
    !{sys.executable} -m pip install nltk==3.8.1
    !{sys.executable} -m pip install jq    

    response = subprocess.run(['cat', '/etc/os-release'], capture_output=True)
    response = response.stdout.decode("utf-8")
    
    if install_complex_pdf:
        if "Amazon Linux" in response: ## SageMaker Notebook

            !sudo yum-config-manager --disable centos-extras
            !sudo rpm -Uvh https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
            !sudo yum -y update
            !sudo yum install -y poppler-utils
            !{sys.executable} -m pip install -U lxml
            !{sys.executable} -m pip install -U kaleido
            !{sys.executable} -m pip install -U uvicorn
            !{sys.executable} -m pip install -U pandas
            !{sys.executable} -m pip install -U numexpr
            !sudo sh install_tesseract.sh
            !sudo amazon-linux-extras install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]"
            !sudo rm -rf leptonica-1.84.1 leptonica-1.84.1.tar.gz tesseract-5.3.1 5.3.1.tar.gz

        else: ## SageMaker Studio
            !sudo apt-get install software-properties-common -y
            !sudo add-apt-repository ppa:alex-p/tesseract-ocr5 -y
            !sudo apt-get update -y
            !sudo apt-get install poppler-utils tesseract-ocr -y
            !sudo apt-get install libgl1-mesa-glx libglib2.0-0 -y
            !sudo apt install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]==0.13.2"

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-core 0.11.14 requires nltk>3.8.1, but you have nltk 3.8.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 52.5 MB/s eta 0:00:00


## 4. Check setting 
Model List <BR>
![nn](../imgs/check.png)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [3]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [4]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.3.1
boto3 version check: 1.35.28
botocore version check: 1.35.28
awscli version check: 1.34.28


In [5]:
bedrock_region=boto3.Session().region_name

In [6]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [7]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-Embeddings-V2': 'amazon.titan-embed-text-v2:0',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Express': 'amazon.titan-text-express-v1',
 'T

## 5. Generation

In [8]:
import base64
from textwrap import dedent
from langchain_aws import ChatBedrock, BedrockLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

### Titan

In [9]:
llm = BedrockLLM(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [10]:
prompt = "Please let us know Amazon Bedrock's advantages in 100 words"

In [11]:
print (colored(llm(prompt), "green"))

/tmp/ipykernel_40587/3721854035.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print (colored(llm(prompt), "green"))



Amazon Bedrock is a cloud-based platform that provides developers with the tools and services they need to build, deploy, and manage applications in the cloud. It offers a wide range of services, including compute, storage, databases, machine learning, and more, and is designed to be highly scalable, reliable, and secure. Amazon Bedrock also provides a variety of developer tools and services, such as the AWS SDK, AWS Lambda, and Amazon S3, that make it easy to build and deploy applications. Additionally, Amazon Bedrock is integrated with other AWS services, such as Amazon EC2, Amazon EKS, and Amazon S3, which allows developers to build and deploy applications in a variety of environments.


### Claude v3.5

In [12]:
llm = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens": 1024,
        "temperature":0,
        "top_p":0.9,
        "stop_sequences": ["\n\nHuman"],
    },
    #streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f64840d7e50>], client=<botocore.client.BedrockRuntime object at 0x7f64961397e0>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'max_tokens': 1024, 'temperature': 0, 'top_p': 0.9, 'stop_sequences': ['\n\nHuman']})

In [13]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are a master answer bot designed to answer user's questions. I'm going to ask you a question."),
        ("human", [{
            "type": "text",
            "text": dedent(
                """
                Here is the question: <question>{question}</question>
                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts". 
                """
                )
            }]
        )
    ]
)

In [14]:
chain = prompt | llm | StrOutputParser()

In [15]:
stream = chain.stream(
    {
        "question": "세이지메이커의 장점을 100단어로 알려주세요."
    }
)
response = ""
for chunk in stream:
    response += chunk
    

SageMaker의 주요 장점을 100단어로 요약하면 다음과 같습니다:

SageMaker는 AWS에서 제공하는 완전 관리형 기계학습 플랫폼으로, 데이터 준비부터 모델 개발, 학습, 배포까지 전체 ML 워크플로우를 지원합니다. 사용자 친화적인 인터페이스로 초보자도 쉽게 사용할 수 있으며, 자동화된 ML 기능으로 생산성을 높일 수 있습니다. 다양한 내장 알고리즘과 프레임워크를 제공하여 개발 시간을 단축시키고, 확장성이 뛰어나 대규모 데이터셋 처리가 가능합니다. 또한 보안과 규정 준수 기능이 내장되어 있어 안전한 환경에서 ML 프로젝트를 수행할 수 있습니다. 비용 효율적인 가격 정책으로 리소스 사용을 최적화할 수 있어 기업의 ML 도입을 가속화합니다.

In [16]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are a master answer bot designed to answer user's questions. I'm going to ask you a question."),
        ("human", [{
            "type": "text",
            "text": dedent(
                """
                세이지메이커의 장점을 100단어로 알려주세요.
                """
                )
            }]
        ),
        ("ai", "SageMaker는 AWS의 완전 관리형 기계학습 플랫폼으로, 데이터 준비부터 모델 배포까지 전체 ML 워크플로우를 간소화합니다. 내장된 알고리즘과 프레임워크를 제공하여 빠른 개발이 가능하고, 자동 하이퍼파라미터 튜닝으로 모델 성능을 최적화합니다. 유연한 노트북 환경과 분산 학습 기능으로 대규모 데이터 처리가 용이하며, 서버리스 추론으로 비용 효율적인 배포가 가능합니다. 또한 MLOps 도구를 통해 모델 관리와 모니터링을 지원하고, 보안 및 규정 준수 기능을 제공합니다. 통합된 개발 환경으로 생산성을 높이고 시간과 비용을 절감할 수 있어 기업의 AI/ML 프로젝트 구현을 가속화합니다."),
        ("human", [{
            "type": "text",
            "text": dedent(
                """
                {command}
                """
                )
            }]
        ),
    ]
)

In [17]:
chain = prompt | llm | StrOutputParser()
stream = chain.stream(
    {
        "command": "궁서체로 진짖하게 말해줘."
    }
)
response = ""
for chunk in stream:
    response += chunk

ERROR:root:Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModelWithResponseStream operation (reached max retries: 4): Too many requests, please wait before trying again.


ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModelWithResponseStream operation (reached max retries: 4): Too many requests, please wait before trying again.